## 의사결정 시뮬레이션 : 리스트 비용 최적화

### 1. 분석 배경
탐지 모델은 통계적으로 1종 오류(선량한 판매자 오인)와 2종 오류(악성 판매자 미탐지)를 완벽하게 배제할 수 없습니다. 
본 시뮬레이션은 각 오류가 발생했을 때의 경제적 손실을 산출하고, 이를 바탕으로 최적의 대응 전략(즉시 차단 vs 선제적 경고)을 도출하는 것을 목적으로 합니다.

### 2. 주요 비용 항목 (Cost Factors)

| 항목 | 용어 | 설명 |
| :--- | :--- | :--- |
| 미탐지 비용 | False Negative | 실제 악성 판매자를 놓쳐 발생하는 사고 피해액 (브랜드 가치 하락, 과징금 등) |
| 오탐지 비용 | False Positive | 선량한 판매자를 악성으로 오해하여 차단했을 때의 손실 (수수료 수익 중단, 보상 비용 등) |
| 선제 경고 비용 | Warning Cost | 의심 대상자에게 소명 기회를 제공하고 관리하는 데 드는 시스템/행정 운영 비용 |

### 3. 분석 가설
> **"확신이 부족한 구간에서는 즉시 차단보다 '선제적 경고 및 소명' 절차를 운영하는 것이 전체 리스크 비용을 최소화한다."**

* 전략 A (현상 유지): 사고 발생 시 사후 대응 중심
* 전략 B (강력 대응): 의심 즉시 차단 중심 (1종 오류 리스크 상존)
* 전략 C (기획 제안): 의심 시 선제적 경고 및 소명 절차 도입 (효율적 투자)

In [ ]:
# 라이브러리 로드
import pandas as pd

# 결과 출력을 소수점 없이 깔끔하게 보여주기 위한 설정
pd.options.display.float_format = "{:,.0f}".format

In [ ]:
def compare_strategy_dataframe(
    n_sellers: int,
    prob_malicious: float,
    error_rate_1: float,
    error_rate_2: float,
    loss_malicious: int,
    loss_false_alarm: int,
    cost_warning: int,
) -> pd.DataFrame:
    # 1. 대상자 분류
    n_malicious = n_sellers * prob_malicious
    n_normal = n_sellers * (1 - prob_malicious)

    # 2. 케이스별 인원 산출
    missed_malicious = n_malicious * error_rate_2
    false_alarms = n_normal * error_rate_1
    warning_cases = n_malicious + false_alarms  # 악성 전체 + 오해받은 정상인

    # 3. 전략별 비용 계산
    cost_missed = missed_malicious * loss_malicious
    cost_false_alarm = false_alarms * loss_false_alarm
    cost_warning_total = warning_cases * cost_warning

    data = {
        "구분": [
            "악성 판매자 미탐지 피해",
            "선량한 판매자 오인 차단 손실",
            "선제적 경고/소명 운영비",
        ],
        "인원(명)": [missed_malicious, false_alarms, warning_cases],
        "단가(원)": [loss_malicious, loss_false_alarm, cost_warning],
        "총 리스크 비용(원)": [cost_missed, cost_false_alarm, cost_warning_total],
    }

    df = pd.DataFrame(data)

    # 결과 요약 계산
    current_total = cost_missed + cost_false_alarm
    print(f"✅ 기존 방식(차단/방치) 총 비용: {current_total:,.0f}원")
    print(f"✅ 기획안(선제적 경고) 총 비용: {cost_warning_total:,.0f}원")
    print(f"🚀 비용 절감액: {current_total - cost_warning_total:,.0f}원")

    return df

result_df = compare_strategy_dataframe(
    n_sellers=10000,
    prob_malicious=0.05,
    error_rate_1=0.03,
    error_rate_2=0.10,
    loss_malicious=200000000,
    loss_false_alarm=10000000,
    cost_warning=200000,
)

result_df